<a href="https://colab.research.google.com/github/lahirunie-dulsara/EN3150-Assignment-3-CNN/blob/Sakith/RealWaste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import zipfile, os, re, shutil
from PIL import Image

zip_path = "/content/drive/MyDrive/Real Waste.zip"

extract_path = "/content/extracted_zip"
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("✅ Unzipped to:", extract_path)


✅ Unzipped to: /content/extracted_zip


In [7]:
base_dir = "/content/extracted_zip/realwaste-main/RealWaste"
for folder in os.listdir(base_dir):
    count = len(os.listdir(os.path.join(base_dir, folder)))
    print(f"{folder}: {count} images")

Miscellaneous Trash: 495 images
Paper: 500 images
Textile Trash: 318 images
Metal: 790 images
Vegetation: 436 images
Cardboard: 461 images
Food Organics: 411 images
Plastic: 921 images
Glass: 420 images


In [10]:
import splitfolders  # install with: pip install split-folders

input_folder = "/content/extracted_zip/realwaste-main/RealWaste"
output_folder = "/content/extracted_zip/realwaste-main/realwaste_split"

splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(.7, .15, .15))

Copying files: 4752 files [00:08, 560.34 files/s]


In [12]:
base_dir = "/content/extracted_zip/realwaste-main/realwaste_split"

# Count images in each subfolder
for split in ['train', 'val', 'test']:
    split_path = os.path.join(base_dir, split)
    print(f"\n📁 {split.upper()} SET")
    total = 0
    for cls in os.listdir(split_path):
        cls_path = os.path.join(split_path, cls)
        count = len(os.listdir(cls_path))

        total += count
        print(f"  {cls}: {count} images")
    print(f"  ➜ Total {split}: {total} images")


📁 TRAIN SET
  Miscellaneous Trash: 346 images
  Paper: 350 images
  Textile Trash: 222 images
  Metal: 553 images
  Vegetation: 305 images
  Cardboard: 322 images
  Food Organics: 287 images
  Plastic: 644 images
  Glass: 294 images
  ➜ Total train: 3323 images

📁 VAL SET
  Miscellaneous Trash: 74 images
  Paper: 75 images
  Textile Trash: 47 images
  Metal: 118 images
  Vegetation: 65 images
  Cardboard: 69 images
  Food Organics: 61 images
  Plastic: 138 images
  Glass: 63 images
  ➜ Total val: 710 images

📁 TEST SET
  Miscellaneous Trash: 75 images
  Paper: 75 images
  Textile Trash: 49 images
  Metal: 119 images
  Vegetation: 66 images
  Cardboard: 70 images
  Food Organics: 63 images
  Plastic: 139 images
  Glass: 63 images
  ➜ Total test: 719 images


In [8]:
import os
import torch
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import datasets, transforms

train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.ToTensor(),
])

val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = datasets.ImageFolder("/content/extracted_zip/realwaste-main/realwaste_split/train", transform=train_transforms)
val_dataset   = datasets.ImageFolder("/content/extracted_zip/realwaste-main/realwaste_split/val", transform=val_test_transforms)
test_dataset  = datasets.ImageFolder("/content/extracted_zip/realwaste-main/realwaste_split/test", transform=val_test_transforms)


In [9]:
import numpy as np

class_counts = np.bincount(dataset.targets)
print("Class counts:", class_counts)

class_weights = 1. / torch.tensor(class_counts, dtype=torch.float)
sample_weights = [class_weights[label] for label in dataset.targets]

Class counts: [461 411 420 790 495 500 921 318 436]
